# Vamos utilizar o mlflow para versionar os nossos modelos 

Este notebook vai ser muito similar ao notebook `10_random_forests_mlflow.ipynb`, com a diferença que aqui vamos registar os nossos modelos no **Model Registry** do mlflow, que nos permite tê los versionados e as suas respectivas experiências e runs trackable.

Referências:
* [Model Registry](https://www.mlflow.org/docs/latest/model-registry.html#concepts)

Para perceber o que muda neste notebook será preciso percorrer o mesmo, e as alterações face ao notebook anterior irão ter um comentário em markdown antes das mesmas

In [1]:
import mlflow

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [3]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "Outcome"

## Utilizar o sqlite para fazer track das experiências

A primeira coisa que vamos fazer é mudar onde as experiências vão ser guardadas.

No notebook anterior estavamos a guarda las no local filesystem na pasta mlruns.

**Para conseguirmos utilizar o model registry** já não vamos poder fazer isto, **temos que utilizar uma base de dados para guardar as experiências, como é referido na [documentação do mlflow](https://www.mlflow.org/docs/latest/tracking.html#backend-stores)**.

O mlflow vem logo com suporte para utilizar o sqlite como database engine, e é o que iremos usar. 

Portanto vamos ter que mudar o `set_tracking_uri` para nos apontar para o ficheiro de database criado pelo sqlite corrido pelo mlflow. 

Nós ainda não temos nenhum ficheiro com uma DB mas isso não é um problema. O mlflow vai criar esse ficheiro com uma DB por nós, basta especificarmos o nomde do ficheiro de db que queremos crirar e ele no set vai cria-lo. 

Tendo em conta isto, no comando abaixo estamos a fazer set para a DB que deveria estar em `../../mlruns/mlflow.db` mas como este ficheiro não existe o mlflow vai criar uma nova db e guarda la nesse ficheiro.

**Nota:** A db, caso não exista, não irá ser criado logo quando executam o comando abaixo, mas apenas quando criam uma experiência nessa db. 

Referências:
* [Armazenamento dos dados da experiência](https://www.mlflow.org/docs/latest/tracking.html#backend-stores)

In [4]:
mlflow.set_tracking_uri("sqlite:///../../mlruns/mlflow.db")

## Criar ou reutilizar uma experiência

Vamos criar uma nova experiência agora que temos uma nova maneira de dar track às experiências.

Não vamos conseguir utilizar a mesma experiência guardada no filesystem e na DB. **A experiência que guardamos no notebook anterior não está visivel na DB que criamos já que utilizamos outro mecanismo para a guardar.**

Vamos especificar o `artifact_location` para que esta experiências não guarde os seus artefactos em `.\mlruns` (na pasta dos notebooks) mas sim em `..\..\mlruns` (na root do projecto) 

**Nota:** Se já correram o notebook equivalente para a logistic regression então a experiment já foi criada lá, pelo que aqui só precisam de fazer o set dessa experiência

In [5]:
from pathlib import Path

artifact_location = Path("../../mlruns/db")

# criar a pasta ../../mlruns/db caso ela não exista
artifact_location.mkdir(parents=True, exist_ok=True)

try:
    mlflow.create_experiment(name="Diabetes Prediction Experiment", artifact_location=artifact_location.as_posix())
except mlflow.MlflowException:
    # experiência já foi criada, só precisamos de fazer set dela
    pass

mlflow.set_experiment(experiment_name="Diabetes Prediction Experiment")

<Experiment: artifact_location=('file:///c:/Users/gilso/OneDrive/Área de '
 'Trabalho/rumos/notebooks/random_forest/../../mlruns/db'), creation_time=1701107536034, experiment_id='1', last_update_time=1701107536034, lifecycle_stage='active', name='Diabetes Prediction Experiment', tags={}>

## Criar os datasets

In [6]:
train_path = ROOT_PATH + 'diabetes_train.csv'
test_path = ROOT_PATH + 'diabetes_test.csv'

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

In [7]:
train_dataset = mlflow.data.from_pandas(train_set, source=train_path, targets=TARGET_COL, name="Diabetes Train Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source=test_path, targets=TARGET_COL, name="Diabetes Test Dataset")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.ap

## Criar uma run

In [8]:
mlflow.start_run(run_name="Random Forest Run")

<ActiveRun: >

## Guardar datasets, modelos, artefactos, métricas e parametros da run - e **registar modelo**

In [9]:
# Guardamos a SEED utilizado como parametro

mlflow.log_param("seed", SEED)

42

In [10]:
# Neste ponto guardarmos o dataset de treino e de teste associado à run

mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)


In [11]:
X_train = train_set.drop([TARGET_COL], axis = 1)
y_train = train_set[TARGET_COL]

X_test = test_set.drop([TARGET_COL], axis = 1)
y_test = test_set[TARGET_COL]

In [12]:
scaler = StandardScaler()

features_names = X_train.columns

X_train[features_names] = scaler.fit_transform(X_train)
X_test[features_names] = scaler.transform(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [13]:
# Vamos querer guardar também o scaler que usamos já que é uma parte vital do pre processamento de dados
# e sem ele não conseguimos reproduzir os resultados
# Como o StandardScaler é um modelo de sklearn vamos usar o mlflow.sklearn.log_model para o guardar

mlflow.sklearn.log_model(scaler, "std_scaler_v2")

2023/11/27 17:53:11 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
c:\Users\gilso\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [14]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,-0.275837,1.501997,0.424428,1.171912,0.818842,0.202551,1.388551,-0.256564
1,0.597861,0.280560,0.525791,1.045640,2.323197,1.783641,-0.696657,0.975594
2,0.306628,-0.659008,0.221703,-1.290397,-0.685512,-0.434985,-0.938791,1.139881
3,-0.858303,0.437154,-1.197374,-0.090810,-0.010585,-0.447736,0.488379,-0.995858
4,0.306628,0.374516,1.944867,-1.290397,-0.685512,2.115160,-0.727992,0.236299


In [15]:
rf = RandomForestClassifier(random_state = SEED).fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [16]:
mlflow.sklearn.log_model(rf, artifact_path="random_forest")

In [17]:
y_preds = rf.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [18]:
accuracy_score(y_test, y_preds)

0.7368421052631579

In [19]:
feature_imp = pd.Series(rf.feature_importances_, index = X_train.columns).sort_values(ascending = False)
feature_imp

Glucose                     0.258462
BMI                         0.180059
Age                         0.144415
DiabetesPedigreeFunction    0.106236
BloodPressure               0.090174
Pregnancies                 0.078640
SkinThickness               0.072315
Insulin                     0.069700
dtype: float64

In [20]:
feature_imp_path = "../../data/feature_importance.csv"
feature_imp.to_csv(feature_imp_path)
mlflow.log_artifact(feature_imp_path)

In [21]:
rf = RandomForestClassifier(random_state = SEED)

parameters = {'n_estimators': [10, 100, 300]}

clf = GridSearchCV(rf, parameters, cv = 5).fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version

In [22]:
mlflow.sklearn.log_model(clf, artifact_path="grid_search_cv")

In [23]:
tunned_rf = clf.best_estimator_

## Tunned Random Forest Model

Não vale a pena registarmos os modelos antes do fine tunning, mas **o modelo tunned definitvamente vamos querer registar !**

Para o fazer, devem adicionar à chamada da função de `log_model` um argumento de **`registered_model_name`** com o nome que querem dar a este modelo. Neste caso vamos chamar ao model "random_forest".

O comando abaixo irá criar a versão 1 deste modelo. Se correrem o mesmo comando mais vez ele irá criar novas versões do modelo. **Em qualquer experiência ou run em que registem um modelo com este mesmo nome ele irá criar uma nova versão do modelo.**

Assim, o Model Registry permite ter uma visão centralizada dos modelos que temos e das suas respectivas versãos, sendo que **cada versão tem associada a si associada a sua run**, pelo que conseguimos perceber o que originou o modelo e qual a sua performance. 

Referências:
* [mflow model api](https://www.mlflow.org/docs/latest/models.html#model-api)
    * [mlflow.sklearn.log_model()](https://www.mlflow.org/docs/latest/python_api/mlflow.sklearn.html#mlflow.sklearn.log_model)

In [24]:
mlflow.sklearn.log_model(tunned_rf, "tunned_rf", registered_model_name="random_forest")

Registered model 'random_forest' already exists. Creating a new version of this model...
Created version '2' of model 'random_forest'.


In [25]:
tunned_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [26]:
mlflow.sklearn.log_model(tunned_rf, "tunned_rf", registered_model_name="random_forester")

Registered model 'random_forester' already exists. Creating a new version of this model...
Created version '2' of model 'random_forester'.


In [27]:
y_preds = tunned_rf.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [28]:
acc = accuracy_score(y_test, y_preds)

In [29]:
mlflow.log_metric("accuracy", acc)

In [30]:
mlflow.end_run()

## Opcional - Ver o modelo registado na UI do mlflow

A UI do mlflow permite ver de forma visual todos os modelos registados até ao momento, qual é a útlima versão do mesmo e a que runs estão associados.

Para correr a UI do mflow, tendo nós o modelo registado na db, temos que especificar esta db como a "backend store" em que temos os metados.

Para correr a UI do mflow é necessário executar, na Anaconda Prompt na raiz deste projeto (pasta rumos) e tendo activo o ambiente utilizado neste projeto, o comando:

`mlflow ui --backend-store-uri sqlite:///./mlruns/mlflow.db`

**Nota:** O comando em cima irá iniciar a UI de mlflow na porta 5000. Caso queiram mudar esta porta devem acrescentar `--port <PORT>` ao comando (em que <PORT> deve ser substituido pela porta desejada). 

O comando acima não irá funcionar caso tenham tido alguns problemas no Windows com a instalação do mlflow, mas como o título desta secção indica este passo é apenas opcional e não irá ser avaliado.

Após executarem este comando, vão poder ver a UI do mlflow no vosso browser acedendo a 

`localhost:5000`

(se tiverem alterado a porta em que o mlflow UI é iniciado então devem de alterar também aqui o 5000 por essa porta)

Na tab de `Models` podem explorar os modelos que registaram.